In [1]:
# training_rcnn.ipynb
import sys
from pathlib import Path
import yaml
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from utils import get_transform, collate_fn
from traffic_dataset import TrafficDataset
from trainer import train_and_validate, prepare_save_dir

c:\Users\Dawid\Desktop\mag\traffic-detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
print(sys.executable)

c:\Users\Dawid\Desktop\mag\traffic-detection\venv\Scripts\python.exe


In [3]:
project_root = Path().resolve()
sys.path.append(str(project_root))

def load_config(path):
    with open(path, 'r') as file:
        return yaml.safe_load(file)

cfg = load_config(project_root / 'cfg' / 'rcnn_day.yaml')
data_cfg = load_config(project_root / 'dataset' / 'dataset_day.yaml')

In [4]:
train_dataset_day = TrafficDataset(
    csv_file='dataset/annotations_train_balanced_day.csv',
    root_dir=Path("dataset/day/train"),
    transforms=get_transform(train=True, cfg=cfg)
)
val_dataset_day = TrafficDataset(
    csv_file='dataset/annotations_test_balanced_day.csv',
    root_dir=Path("dataset/day/test"),
    transforms=get_transform(train=False, cfg=cfg)
)

train_loader_day = DataLoader(train_dataset_day, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=16)
val_loader_day = DataLoader(val_dataset_day, batch_size=cfg['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=16)

c:\Users\Dawid\Desktop\mag\traffic-detection\venv\Lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
save_dir_day = prepare_save_dir(project_root / "runs_rcnn" / "rcnn_day_balanced", cfg)

model_day = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model_day.roi_heads.box_predictor.cls_score.in_features
model_day.roi_heads.box_predictor = FastRCNNPredictor(in_features, data_cfg['nc'])

c:\Users\Dawid\Desktop\mag\traffic-detection\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dawid\Desktop\mag\traffic-detection\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
print(torch.cuda.is_available())
print(torch.version.cuda)
torch.cuda.empty_cache()

True
12.1


In [7]:
train_and_validate(model_day, cfg, data_cfg, train_loader_day, val_loader_day, save_dir_day)

[Epoch 1/50] Training:   0%|          | 0/3194 [01:07<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 5.35 GiB is free. Of the allocated memory 1.60 GiB is allocated by PyTorch, and 50.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# train_dataset_night = TrafficDataset(
#     csv_file='dataset/annotations_train_balanced_night.csv',
#     root_dir=Path("dataset/night/train"),
#     transforms=get_transform(train=True, cfg=cfg)
# )
# val_dataset_night = TrafficDataset(
#     csv_file='dataset/annotations_test_balanced_night.csv',
#     root_dir=Path("dataset/night/test"),
#     transforms=get_transform(train=False, cfg=cfg)
# )

# train_loader_night = DataLoader(train_dataset_night, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=8)
# val_loader_night = DataLoader(val_dataset_night, batch_size=cfg['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=8)

In [ ]:
# save_dir_night = prepare_save_dir(project_root / "runs_rcnn" / "rcnn_night", cfg)

# model_night = fasterrcnn_resnet50_fpn(pretrained=True)
# in_features = model_night.roi_heads.box_predictor.cls_score.in_features
# model_day.roi_heads.box_predictor = FastRCNNPredictor(in_features, data_cfg['nc'])

In [ ]:
# train_and_validate(model_day, cfg, data_cfg, train_loader_night, val_loader_night, save_dir_night)